In [1]:
import numpy as np
import cv2
from fpdf import FPDF 
from docx import Document

In [2]:
class Transformations:
    
    def __init__(self,centres,height,width,img):
        self.centre = centres
        self.h,self.w = height,width
        self.image = img
    
    def rotate(self,deg):
        matrix = cv2.getRotationMatrix2D(self.centre,angle = deg,scale = 1,flags=cv2.INTER_CUBIC,)
        rotated = cv2.warpAffine(self.image,matrix,(self.w,self.h))
        return rotated
        
    def translate(self,Dx,Dy):
        matrix = np.float32([
            [1,0,Dx],[0,1,Dy]
        ])  
        translated = cv2.warpAffine(self.image,matrix,(self.w,self.h))
        return translated
    
    def affline(self):
        pts1 = np.float32([[50,50],[200,50],[50,200]])
        pts2 = np.float32([[10,100],[200,50],[100,250]])
        M = cv2.getAffineTransform(pts1,pts2)
        dst = cv2.warpAffine(self.image,M,(self.w,self.h))
        return dst                  
    
    def skew_correction(self,deg):
        gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        gray = cv2.bitwise_not(gray)
        thresh = cv2.threshold(gray, 0, 255,
        cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        coords = np.column_stack(np.where(thresh > 0))
        angle = cv2.minAreaRect(coords)[-1]
        if angle < -45:
            angle = -(90 + angle)
        else:
            angle = -angle
        (h, w) = self.image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(self.image, M, (w, h),
            flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)    
        return rotated

In [2]:
class conversion():
    
    def __init__(self,text):
        self.doc = text
     
        
    def createPDF(self,name):
        try:
            if len(self.doc)>0:
                pdf = FPDF() 
                pdf.add_page() 
                pdf.set_font("Arial", size = 15) 
                pdf.cell(200, 10, txt = self.doc,  
                         ln = 1, align = 'C') 
                pdf.output(name +".pdf") 
        except:
            pass
        
        
    def createDocx(self,name,title):
        document = Document()
        document.add_heading(title,0)
        p = document.add_paragraph(self.doc)
        document.add_page_break()
        document.save(title + '.docx')